In [1]:
import os
import sys

import numpy as np
import matplotlib.pyplot as plt

import scanpy as sc
import scvi
import torch

from pytorch_lightning.loggers import WandbLogger

import wandb

import session_info
import warnings
from pyprojroot.here import here

from dotenv import load_dotenv

sys.path.insert(1, str(here('bin')))

torch.set_float32_matmul_precision('high')

#plt.style.use(['science','nature','no-latex'])
dpi_fig_save = 300
sc.set_figure_params(dpi=100, dpi_save=dpi_fig_save, vector_friendly=True)

# Setting some parameters
warnings.filterwarnings("ignore")

overwriteData = True
overwriteFigures = True

In [2]:
assert load_dotenv(here('.env'))

In [3]:
class CustomWandbLogger(WandbLogger):
    @property
    def save_dir(self):
        """Gets the save directory.

        Returns:
            The path to the save directory.

        """
        return self.experiment.dir

In [4]:
scvi.settings.seed = 0
print("Last run with scvi-tools version:", scvi.__version__)

Seed set to 0


Last run with scvi-tools version: 1.1.2


**Load data**

In [5]:
# Load the h5ad file
adata = sc.read_h5ad(here("00_data_processing/results/03_INFLAMMATION_main_normalized_HVGsubset.h5ad"), backed='r')
adata

AnnData object with n_obs × n_vars = 4918140 × 3236 backed at '/home/jupyter/Inflammation-PBMCs-Atlas-R1/00_data_processing/04_scVI_integration/03_INFLAMMATION_main_normalized_HVGsubset.h5ad'
    obs: 'studyID', 'libraryID', 'sampleID', 'chemistry', 'technology', 'patientID', 'disease', 'timepoint_replicate', 'treatmentStatus', 'therapyResponse', 'sex', 'age', 'BMI', 'binned_age', 'diseaseStatus', 'smokingStatus', 'ethnicity', 'institute', 'diseaseGroup', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'total_counts_plt', 'log1p_total_counts_plt', 'pct_counts_plt', 'doublet_score', 'predicted_doublet', 'S_score', 'G2M_score', 'phase'
    var: 'hgnc_id', 'symbol', 'locus_group', 'HUGO_status', 'mt', 'ribo', 'hb', 'plt', 'n_cells_by_co

## scVI integration

#### Parameters

In [6]:
# https://docs.scvi-tools.org/en/stable/api/reference/scvi.train.Trainer.html#scvi.train.Trainer
setup_kwargs = dict(
    layer="counts", 
    batch_key='libraryID', 
    categorical_covariate_keys = ['chemistry', 'studyID', 'disease', 'sampleID'],
    labels_key = 'chemistry'    
)

scvi_kwargs = dict(n_hidden=512,
                   n_latent=30,
                   n_layers=2,
                   gene_likelihood='nb',
                   dispersion='gene-label')

trainer_kwargs = dict(
    checkpointing_monitor = 'elbo_validation',
    early_stopping_monitor = 'reconstruction_loss_validation',
    early_stopping_patience = 10,
    early_stopping_min_delta=0.1,
    early_stopping = True,
    max_epochs = 1000,

    #logger = # wandb
)
# https://docs.scvi-tools.org/en/stable/api/reference/scvi.train.TrainingPlan.html#scvi.train.TrainingPlan
plan_kwargs = dict(
    lr = 5e-4,
    #reduce_lr_on_plateau = True
)

# https://docs.scvi-tools.org/en/stable/api/reference/scvi.module.VAE.html#scvi.module.VAE
#vae = dict(
#    use_layer_norm='both',
#    use_batch_norm='none',
#    encode_covariates=True,
#    deeply_inject_covariates=False
#)
parameter_dict = setup_kwargs | scvi_kwargs | trainer_kwargs | plan_kwargs

In [7]:
parameter_dict

{'layer': 'counts',
 'batch_key': 'libraryID',
 'categorical_covariate_keys': ['chemistry', 'studyID', 'disease', 'sampleID'],
 'labels_key': 'chemistry',
 'n_hidden': 512,
 'n_latent': 30,
 'n_layers': 2,
 'gene_likelihood': 'nb',
 'dispersion': 'gene-label',
 'checkpointing_monitor': 'elbo_validation',
 'early_stopping_monitor': 'reconstruction_loss_validation',
 'early_stopping_patience': 10,
 'early_stopping_min_delta': 0.1,
 'early_stopping': True,
 'max_epochs': 1000,
 'lr': 0.0005}

In [8]:
run_name = f"Step00_COV{'_'.join(parameter_dict['categorical_covariate_keys'])}"
run_name

'Step00_COVchemistry_studyID_disease_sampleID'

In [9]:
# https://docs.scvi-tools.org/en/stable/api/reference/scvi.model.SCVI.html#scvi.model.SCVI.train


In [10]:
scvi.model.SCVI.setup_anndata(adata, 
                              **setup_kwargs)

In [11]:
logger = CustomWandbLogger(name = run_name, project='inflammation_atlas_R1', config = parameter_dict)

Wd decided to use gene-label because-...
https://discourse.scverse.org/t/what-model-to-use-when-integrating-batches-of-scrna-seq-matrices-containing-150-000-t-and-innate-lymphoid-cell-ilc-sub-populations/454/7

In [12]:
model = scvi.model.SCVI(adata, **scvi_kwargs)

In [13]:
model.train(logger=logger, plan_kwargs = plan_kwargs, **trainer_kwargs)

GPU available: True (cuda), used: True


TPU available: False, using: 0 TPU cores


IPU available: False, using: 0 IPUs


HPU available: False, using: 0 HPUs


wandb: Currently logged in as: dav1989 (inflammation). Use `wandb login --relogin` to force relogin


...

Epoch 42/1000:   4%|█▋                                        | 41/1000 [4:41:09<108:17:19, 406.51s/it, v_num=7nr3, train_loss_step=712, train_loss_epoch=715]

Epoch 42/1000:   4%|█▊                                        | 42/1000 [4:47:55<108:09:13, 406.42s/it, v_num=7nr3, train_loss_step=712, train_loss_epoch=715]

Epoch 42/1000:   4%|█▊                                        | 42/1000 [4:47:55<108:09:13, 406.42s/it, v_num=7nr3, train_loss_step=782, train_loss_epoch=715]

Epoch 42/1000:   4%|█▊                                        | 42/1000 [4:47:55<109:27:19, 411.32s/it, v_num=7nr3, train_loss_step=782, train_loss_epoch=715]


Monitored metric reconstruction_loss_validation did not improve in the last 10 records. Best score: 705.487. Signaling Trainer to stop.


In [14]:
wandb.finish()

wandb:                                                                                


wandb: 
wandb: Run history:
wandb:                     elbo_train █▅▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:                elbo_validation █▅▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:                          epoch ▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
wandb:                kl_global_train ▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:           kl_global_validation ▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:                 kl_local_train █▆▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:            kl_local_validation █▅▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:                      kl_weight ▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
wandb:      reconstruction_loss_train █▄▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb: reconstruction_loss_validation █▆▄▄▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:               train_loss_epoch █▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂▂▂▂
wandb:                train_loss_step ▆▄▄▅▃▆▂▄▅▅▅▅▆▄▃█▆▆▄▅▄▆▇▆▆▅▅▄▃▆▄▂▅▅▅▅▃▃▅▁
wandb:            traine

wandb: 🚀 View run Step00_COVchemistry_studyID_disease_sampleID at: https://wandb.ai/inflammation/inflammation_atlas_R1/runs/k6oy7nr3
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)


wandb: Find logs at: ./wandb/run-20240426_225044-k6oy7nr3/logs


## Save the results

In [15]:
if overwriteData:
    model.save(here(f"00_data_processing/results/04_INFLAMMATION_main_HVGsubset_scVI_step00/"), 
               overwrite = True, 
               save_anndata = False)

In [16]:
session_info.show(excludes=['google3'],)